### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [2]:
import numpy as np
import pandas as pd
from library import (
    get_squad_as_index,
    remove_unnamed_cols,
    clean_opp_df,
    clean_main_df,
    merge_dfs,
    get_single_season_league_data,
)


In [3]:
my_list_first =pd.read_html("https://fbref.com/en/comps/9/2021-2022/")
leagues_list = my_list_first.copy()